In [1]:
import random
import pandas as pd

# Cards

Let's make a poker game, or at least part of it. Our goal will ultimately be to create a game of 5 card draw poker. For now, we'll try to create a deck of cards, and hands for the players that inherit from the deck. So, we should have 3 classes for this stage: Card, Deck, and Hand:
<ul>
<li> Card</li>
    <ul>
    <li> A card is one individual card, made up of a Suit and a Rank (ace, two, three...queen, etc...)</li>
    <li> A card needs to provide str, lt, gt, eq to allow for comparisons and printing. </li>
    </ul>
<li> Deck</li>
    <ul>
    <li> A deck is made up of a bunch of cards. </li>
    <li> A deck needs to be able to be initialized - here we want to fill it with 52 cards and potentially shuffle it.</li>
    <li> A deck needs to have a str and len method override. </li>
    <li> A deck needs to be able to deal out cards into smaller decks, of whatever number is requested. </li>
    </ul>
<li> Hand</li>
    <ul>
    <li> The hand class inheirits from the deck class and represents a "mini-deck" of five cards (we're building a 5 card poker deck for now, later on we'll change this
    up a bit to make this into a five_card_poker deck, and that'll allow us to make other decks that inheirit from the Deck class but make sense for different games).</li>
    <li> The hand needs to define a lt method, that can act to do a comparison for determining which had wins, in the context of poker. This can be wrong! We need a system for scoring that functions, we can make it poker-y later. </li>
    </ul>
</ul>

Underneath the spot we have to make the classes, there is a bunch of stuff to test and see if it works is there. This testing is limited, we should try to create more to really test things here - if we are going to build this up into an actual game of poker (and we intend to), we should make sure that things work first, especially the edge cases.

<b>Note:</b> a full game of poker, especially the part of calculating who wins, is actually reasonably complex. That part isn't really critically important, if we make some error in the calculation of which three of a kind tie-breaker <i>actually</i> wins, it isn't a big deal. The important point is that we can create a deck that works - even if there's some tiny error in the exact logic. We can always fix that later In reality, game logic errors are something we'd notice in user testing, where we would be actually playing to make sure it works, and at some point we'd see that the wrong player won some hand, and we'd have to go back and fix it. This is a bit of agile thinking, right now we're in sprint number one. 

### Design Choices

While we make this, there are many choices that we need to make, and many of them don't have a super clear answer. This is normal and OK here. We want to focus on - what does it need to do? Can it do it? It is OK to have something that half works, we can revise. Some things to pay particular attention to are:
<ul>
<li> Where something should "live" - there are lots of actions here that could be done by the deck, or the hand, or the card. We need to think about where it makes sense to put them. </li>
<li> How to represent things - we could represent a card as a string, or as a tuple, or as a class. We could represent a deck as a list of cards, or as a class. We could represent a hand as a list of cards, or as a class. We need to think about what makes sense. </li>
<li> How to do things - we could shuffle a deck by making a new deck and randomly drawing cards from the old deck and putting them in the new deck, or we could shuffle the deck in place. We could sort a hand by making a new hand and drawing cards from the old hand and putting them in the new hand, or we could sort the hand in place. We need to think about what makes sense. </li>
<li> Constructing a new deck - we have different scenarios where a deck will be created, how do we want to handle that? </li>
</ul>

All of these things are somewhat open - we can make a choice that works, go with it, then change it if we encounter something that doesn't work. If our code is relatively modular, with well encapsulated classes and methods, we can change things without too much difficulty. 

### Goal For Now

Right now we want to be able to have a deck, deal some hands out of it, and compare those hands to each other. The comparison part should be based on actual poker logic, but it is ok if that logic is incomplete or wrong - we just need some systemic way to compare hands and order them by their value. 

### Potentially Useful Notes

#### Pop()

An item can be grabbed from a list, and removed from that list, with the pop() method. 

#### Random.shuffle()

Random.shuffle() can be used to shuffle a list to random order. 

#### Make an Iterable

Our deck class should be an iterable - i.e. we should be able to loop through it like we could a list or other container. For a class to work as an iterable we need to define two methods:
<ul>
<li> __iter__ </li>
    <ul>
    <li> Iter returns the iterator, or the object itself. </li>
    </ul>
<li> __next__ </li>
    <ul>
    <li> Next returns the next item. </li>
    </ul>
</ul>

If we implement these two methods, we'll have an item that can work as an iterable. The iter method is simple, we just return the object. The next method is the bulk of the functionality, it needs to always provide the next item in the iterable. If there are no more items, it needs to raise a StopIteration exception. If we have an object that is holding its data in something like a list or tuple, we can do this pretty easily - just ask the existing container for its next item, and raise a StopIteration exception if there are no more items.

In [2]:

class Card():
    """
    Attributes:
        suit: suit of card
        rank: number of card
    Methods:
        init
        str
        repr
        eq
        lt
    """
    suits = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
    ranks = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']
    def __init__(self, suit, rank):
        if suit not in Card.suits:
            raise ValueError(f'{suit} does not exist')
        else:
            self._suit = suit
        if rank not in Card.ranks:
            raise ValueError(f'{rank} does not exist')
        else:
            self._rank = rank

    @property
    def suit(self):
        return self._suit
    
    @property
    def rank(self):
        return self._rank
    
    def __str__(self) -> str:
        return f'{self.rank} of {self.suit}'
    
    def __repr__(self):
        return f'Card: ({self.rank}, {self.suit})'
    
    # note, less than must be coded first
    def __lt__(self, other):
        index_rank_self = Card.ranks.index(self.rank)
        index_rank_other = Card.ranks.index(other.rank)
        index_suit_self = Card.suits.index(self.suit)
        index_suit_other = Card.suits.index(other.suit)        

        if index_rank_self < index_rank_other:
            return True
        elif index_rank_self == index_rank_other:
            if index_suit_self < index_suit_other:
                return True
            else:
                return False
        else:
            return False

    def __eq__(self, other):
        index_rank_self = Card.ranks.index(self.rank)
        index_rank_other = Card.ranks.index(other.rank)
        index_suit_self = Card.suits.index(self.suit)
        index_suit_other = Card.suits.index(other.suit)       
        return index_suit_self == index_suit_other and  index_rank_self == index_rank_other



Test the card a bit...

In [3]:
card_1 = Card('Hearts', 'Two')
card_2 = Card('Hearts', 'Seven')
print(card_1)
print(card_2)

Two of Hearts
Seven of Hearts


In [4]:
print(card_2 < card_1)
print(card_1 < card_2)

False
True


In [5]:
card_3 = Card('Diamonds', 'Two')
print(card_1 == card_3)
print(card_1 < card_3)

False
False


In [14]:
import random

class Deck():
    def __init__(self, *cards, shuffle = True, populate = True):
        self._deck = []
        
        # creating deck would take priority over cards
        if populate:
           self.createDeck()
        elif len(cards) > 0:
            for card in cards:
                self.deck.append(card)        
        if shuffle:
            self.shuffle()


    @property
    def deck(self):
        return self._deck
    
    def shuffle(self):
        random.shuffle(self._deck)

    def addCard(self, card):
        self.deck.append(card)

    def dealCard(self):
        '''
        Returns:
            'top' card of _deck
        '''
        return self.deck.pop()
    
    def dealHands(self, hands = 4, cards = 5) -> list:
        dealHands = []
        numCards = len(self._deck)

        # ensure enough cards are made
        if numCards >= cards * hands:
            for i in range(hands):
                # hand = []
                hand = Hand(size = cards)
                for j in range(cards):
                    hand.addCard(self.dealCard())
                dealHands.append(hand)
        else:
            print("Not enough cards to distribute")
        return dealHands
    
    def isInDeck(self, card):
        return card in self._deck
    

    def createDeck(self):
        for suit in Card.suits:
            for rank in Card.ranks:
                self._deck.append(Card(suit, rank))
    
    def __str__(self):
        return f'Deck of {len(self.deck)} cards'
    
    def __repr__(self):
        ret_string = 'Deck has ' + str(len(self.deck)) + ' cards\n'
        for card in self.deck:
            ret_string += card.__str__() + '\n'
        return f'Deck of {len(self.deck)} cards\n{ret_string}'

    def __len__(self):
        return len(self._deck)
    
    def __iter__(self):
        self.index = 0
        return self
    
    def __next__(self):
        index = self.index
        if index >= len(self.deck):
            raise StopIteration
        
        self.index += 1
        return self.deck[index]

Test the Deck a bit...

In [7]:
deck1 = Deck(populate=True)
print(deck1)
print(len(deck1))

for card in deck1:
    print(card)

Deck of 52 cards
52
Three of Diamonds
Eight of Clubs
Queen of Diamonds
King of Diamonds
Nine of Hearts
Nine of Diamonds
Four of Hearts
Jack of Clubs
Ace of Diamonds
Queen of Hearts
Five of Spades
Two of Hearts
Eight of Spades
Ten of Spades
Nine of Clubs
Six of Clubs
Queen of Spades
Ace of Spades
Eight of Diamonds
Ten of Clubs
Jack of Diamonds
Four of Spades
Six of Spades
King of Clubs
Five of Diamonds
Queen of Clubs
King of Hearts
Six of Diamonds
Three of Clubs
Two of Diamonds
Jack of Spades
King of Spades
Nine of Spades
Seven of Hearts
Five of Clubs
Two of Spades
Five of Hearts
Six of Hearts
Three of Spades
Ten of Hearts
Eight of Hearts
Four of Diamonds
Seven of Spades
Seven of Clubs
Two of Clubs
Seven of Diamonds
Ten of Diamonds
Ace of Clubs
Three of Hearts
Jack of Hearts
Four of Clubs
Ace of Hearts


In [8]:
test_deck = Deck(populate=True)
# print(test_deck)
# print(repr(test_deck))
test_deck.shuffle()
print(repr(test_deck))


Deck of 52 cards
Deck has 52 cards
Nine of Diamonds
King of Diamonds
Ace of Clubs
Queen of Spades
Queen of Diamonds
Six of Diamonds
Seven of Diamonds
Ten of Clubs
Eight of Spades
Jack of Spades
Ace of Spades
Ten of Spades
Seven of Hearts
Eight of Clubs
King of Spades
Nine of Clubs
Four of Diamonds
Five of Hearts
Ten of Diamonds
Six of Hearts
Nine of Hearts
Two of Diamonds
Three of Diamonds
Queen of Hearts
Ten of Hearts
Three of Spades
Seven of Spades
Queen of Clubs
Four of Clubs
Six of Clubs
Three of Clubs
Seven of Clubs
Six of Spades
Jack of Diamonds
Four of Hearts
Two of Spades
Five of Diamonds
Two of Hearts
Five of Spades
Two of Clubs
King of Hearts
Ace of Diamonds
King of Clubs
Eight of Hearts
Jack of Hearts
Eight of Diamonds
Four of Spades
Nine of Spades
Jack of Clubs
Ace of Hearts
Five of Clubs
Three of Hearts



In [15]:
class Hand(Deck):
    hands = ['High Card']
    def __init__(self, *cards, size = 5):
        super().__init__(*cards, populate=False)
        # max size of hand
        self._size = size
    
    @property
    def size(self):
        return self._size

    def addCard(self, card):
        if len(self.deck) < self.size:
            super().addCard(card)

    def sortHand(self):
        self.deck.sort()

    def getScore(self):
        score = self.checkHand()
        return score
        
    def checkHand(self):
        if self.checkRoyalFlush():
            return 9
        elif self.checkStraightFlush():
            return 8
        elif self.checkQuads():
            return 7
        elif self.checkFullHouse():
            return 6
        elif self.checkFlush():
            return 5
        elif self.checkStraight():
            return 4
        elif self.checkTrips():
            return 3
        elif self.checkTwoPair():
            return 2
        elif self.checkPair():
            return 1
        else:
            return 0

    def checkFlush(self):
        suit = self.deck[0].suit
        for card in self.deck:
            if card.suit != suit:
                return False
        return True
    def checkStraight(self):
        rank = self.deck[0].rank
        for card in self.deck:
            if card.rank != rank:
                return False
        return True
    def checkPair(self):
        rank = self.deck[0].rank
        for card in self.deck:
            if card.rank == rank:
                return True
        return False
    def checkTrips(self):
        rank = self.deck[0].rank
        count = 0
        for card in self.deck:
            if card.rank == rank:
                count += 1
        return count == 3
    def checkQuads(self):
        rank = self.deck[0].rank
        count = 0
        for card in self.deck:
            if card.rank == rank:
                count += 1
        return count == 4
    def checkFullHouse(self):
        rank = self.deck[0].rank
        count = 0
        for card in self.deck:
            if card.rank == rank:
                count += 1
        return count == 3
    def checkTwoPair(self):
        rank = self.deck[0].rank
        count = 0
        for card in self.deck:
            if card.rank == rank:
                count += 1
        return count == 2
    def checkStraightFlush(self):
        return self.checkStraight() and self.checkFlush()
    def checkRoyalFlush(self):
        return self.checkStraightFlush() and self.deck[0].rank == 'Ace'
    

    def isRoyalFlush(self):
        for card in self.deck:
            return
        pass

    #comparisons
    def __lt__(self, other):
        self_score = self.checkHand()
        other_score = self.checkHand()
        if self_score < other_score:
            return True
        elif self_score > other_score:
            return False
        # elif self_score == other_score:
        #     #tie breakers
    #print hand


Test the Hand a bit...

In [16]:
deck2 = Deck(populate=True)
hands = deck2.dealHands(4,5)


In [17]:
hand1 = hands[0]
hand1.sortHand()
# print(hand1)
print(repr(hand1))


Deck of 5 cards
Deck has 5 cards
Five of Clubs
Six of Clubs
Six of Spades
Seven of Spades
Jack of Hearts



## Testing

We should make sure that this works. 

##### Create Deck

Make a new deck and see what's in it. We want one that is ready to play, so we'll want it shuffled and filled with 52 cards. 

In [20]:
d = Deck(shuffle=True, populate=True)
test_hands = list(map(Hand.deckToHand, deck1.deal(4,5)))

AttributeError: type object 'Hand' has no attribute 'deckToHand'

##### Deal Hands, Check Score

Now we should check the dealing of hands - a big one, I checked this a bunch of times while making it, running it over and over. Right now, I'm going to check for a few things:
<ul>
<li> The deal produces the expected results - 4 hands of 5 <i>different</i> cards each. </li>
<li> I'll score each of the cards, based on the score logic defined in the Hand class. </li>
<li> Below this part, I want to make sure that the cards that were dealt into hands here aren't still in the deck. </li>
    <ul>
    <li> We could have built something that tested the deal by making a deck, dealing cards, looking at the cards dealt and left, and verifying programatically that the cards dealt are no longer in the deck. </li>
    <li> <b>Note for Sanity:</b> when we are testing this we need to make sure that which cells we've run is consistent. In mine, we make a deck of 52, then deal from it. If we were to accidentally do extra (or fewer) deals, refresh the deck, or other things before we check the results, we can easily break things. </li>
    </ul>
</ul>

<b>Note:</b> the list(map) thing is because map returns a map iterator, not a list. We can convert it to a list to see the results.

In [19]:
hands = list(map(Hand.addCard, d.deal(num_hands=4, card_per_hand=5)))

for hand in hands:
    print(hand.checkHand())
    print(hand)

NameError: name 'd' is not defined

In [457]:
print(d)

0: Four of Hearts
1: Ace of Hearts
2: Three of Clubs
3: Eight of Diamonds
4: Five of Diamonds
5: Three of Diamonds
6: Seven of Diamonds
7: Ten of Diamonds
8: Jack of Hearts
9: Six of Diamonds
10: Four of Diamonds
11: Ace of Diamonds
12: Eight of Spades
13: King of Spades
14: Five of Clubs
15: Four of Spades
16: Two of Hearts
17: Four of Clubs
18: Jack of Diamonds
19: Queen of Diamonds
20: Eight of Clubs
21: Two of Clubs
22: Nine of Hearts
23: Ten of Clubs
24: Queen of Hearts
25: King of Diamonds
26: Six of Clubs
27: Six of Hearts
28: Ten of Hearts
29: Two of Spades
30: Seven of Spades
31: Ace of Clubs



##### Sort and Check Again

Here, we want to sort by the (rough) score of the hand, and see if the hands are in the right order. We can do this by sorting the hands by their score, and then checking the order. This checks the sorting logic, and the scoring logic.

In [21]:
hands.sort(reverse=True)
for hand in hands:
    print(hand.checkHand())
    print(hand)

1
Deck of 5 cards
1
Deck of 5 cards
1
Deck of 5 cards
1
Deck of 5 cards


##### Deal Another Set of Hands

Here, we want to deal another set of hands, and check that the cards are different. We can do this by checking that the cards in the new hands are not in the old hands, and vice versa. We also want to print the deck again, and make sure that it keeps shrinking as we deal from it. 

In [459]:
hands2 = list(map(Hand.deckToHand, d.deal(num_hands=7, card_per_hand=2)))

In [460]:
for hand in hands2:
    print(hand.checkHand())
    print(hand)

1
0: Seven of Spades
1: Ace of Clubs

1
0: Ten of Hearts
1: Two of Spades

4
0: Six of Clubs
1: Six of Hearts

1
0: King of Diamonds
1: Queen of Hearts

1
0: Ten of Clubs
1: Nine of Hearts

5
0: Two of Clubs
1: Eight of Clubs

5
0: Jack of Diamonds
1: Queen of Diamonds



In [461]:
print(d)

0: Four of Hearts
1: Ace of Hearts
2: Three of Clubs
3: Eight of Diamonds
4: Five of Diamonds
5: Three of Diamonds
6: Seven of Diamonds
7: Ten of Diamonds
8: Jack of Hearts
9: Six of Diamonds
10: Four of Diamonds
11: Ace of Diamonds
12: Eight of Spades
13: King of Spades
14: Five of Clubs
15: Four of Spades
16: Two of Hearts
17: Four of Clubs



##### Other Card Tests

This is more from development than testing that we still need, but it won't hurt. Here I'm testing the creation of cards and hands manually, as well as the printout of the short form. 

In [462]:
card1 = Card("Hearts", "Queen")
card2 = Card("Spades", "Queen")
card3 = Card("Hearts", "Seven")
card4 = Card("Spades", "Seven")
card5 = Card("Hearts", "Nine")

hand1 = Hand(card1, card2, card3, card4, card5)
print(hand1)

0: Queen of Spades
1: Seven of Spades
2: Queen of Hearts
3: Nine of Hearts
4: Seven of Hearts



In [463]:
hand1.removeCard("Hearts", "Seven")
print(hand1)

0: Queen of Spades
1: Seven of Spades
2: Queen of Hearts
3: Nine of Hearts



In [464]:
print(Deck.tooShort(card1))
print(Deck.tooShort(card3))

QH
7H
